In [65]:
import re


def parseMonkeys(filename="11.input"):
    monkeys = {}
    with open(filename, "r") as f:
        data = f.readlines()
        for i in range(0, len(data), 7):
            m = re.match(r"Monkey (?P<monkeyIndex>\d+):", data[i].strip())
            monkeyIndex = int(m.group("monkeyIndex"))
            m = re.match(r"Starting items: (?P<items>.*)", data[i + 1].strip())
            items = m.group("items").split(", ")
            items = [int(item) for item in items]
            m = re.match(r"Operation: new = (.*)", data[i + 2].strip())
            operation = m.group(1)
            m = re.match(r"Test: divisible by (?P<divisor>\d+)", data[i + 3].strip())
            divisor = int(m.group("divisor"))
            m = re.match(
                r"If true: throw to monkey (?P<monkeyA>\d+)", data[i + 4].strip()
            )
            monkeyA = int(m.group("monkeyA"))
            m = re.match(
                r"If false: throw to monkey (?P<monkeyB>\d+)", data[i + 5].strip()
            )
            monkeyB = int(m.group("monkeyB"))
            monkeys[monkeyIndex] = {
                "items": items,
                "operation": operation,
                "divisor": divisor,
                "monkeyA": monkeyA,
                "monkeyB": monkeyB,
            }
    return monkeys


In [67]:
from collections import Counter
from functools import reduce

rounds = 20
monkeyInspectionCounts = Counter()

monkeys = parseMonkeys()
for _ in range(rounds):
    for i in monkeys:
        operation = lambda old: eval(monkeys[i]["operation"])
        items = [*monkeys[i]["items"]]
        monkeys[i]["items"] = []
        for worryLevel in items:
            monkeyInspectionCounts[i] += 1
            worryLevel = operation(worryLevel) // 3
            if worryLevel % monkeys[i]["divisor"] == 0:
                monkeys[monkeys[i]["monkeyA"]]["items"].append(worryLevel)
            else:
                monkeys[monkeys[i]["monkeyB"]]["items"].append(worryLevel)

print(monkeyInspectionCounts)
print(reduce(lambda x, t: x * t[1], monkeyInspectionCounts.most_common(2), 1))


Counter({5: 340, 3: 333, 4: 328, 1: 323, 2: 308, 6: 43, 7: 34, 0: 9})
113220


In [69]:
rounds = 10000
monkeyInspectionCounts = Counter()

monkeys = parseMonkeys()
# all divisors are prime
divisorProduct = reduce(lambda x, m: x * m["divisor"], monkeys.values(), 1)

for _ in range(rounds):
    for i in monkeys:
        operation = lambda old: eval(monkeys[i]["operation"])
        items = [*monkeys[i]["items"]]
        monkeys[i]["items"] = []
        for worryLevel in items:
            monkeyInspectionCounts[i] += 1
            # for all divisors, (worryLevel mod divisorProduct) is congruent to worryLevel modulo divisor
            # (worryLevel % divisorProduct) % divisor equals to worryLevel % divisor, so we can throw
            # (worryLevel % divisorProduct) to the next monkey
            worryLevel = operation(worryLevel) % divisorProduct
            if worryLevel % monkeys[i]["divisor"] == 0:
                monkeys[monkeys[i]["monkeyA"]]["items"].append(worryLevel)
            else:
                monkeys[monkeys[i]["monkeyB"]]["items"].append(worryLevel)

print(monkeyInspectionCounts)
print(reduce(lambda x, t: x * t[1], monkeyInspectionCounts.most_common(2), 1))


Counter({3: 175703, 2: 174155, 1: 172759, 5: 171471, 4: 169190, 0: 20133, 7: 11362, 6: 3022})
30599555965
